<a href="https://colab.research.google.com/github/firemoraster/Minimum-area-triangle-from-a-given-set-of-points/blob/main/triangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output

# Функція для обчислення площі трикутника
def calculate_triangle_area(triangle):
    p1, p2, p3 = triangle[:, 0, :]
    area = 0.5 * abs(p1[0]*(p2[1]-p3[1]) + p2[0]*(p3[1]-p1[1]) + p3[0]*(p1[1]-p2[1]))
    return area

# Функція для малювання трикутника та опуклої оболонки
def draw_enclosing_triangle_and_hull(points):
    if points.size == 0:
        print("Список точок порожній. Будь ласка, додайте точки для обробки.")
        return None, None

    ret, triangle = cv2.minEnclosingTriangle(points)
    if ret == 0:
        print("Неможливо знайти мінімальний трикутник для цього набору точок.")
        return None, None

    # Графічне зображення
    plt.figure(figsize=(8, 8))

    # Малювання точок
    plt.scatter(points[:, 0], points[:, 1], s=50, c='red', label='Points')

    # Опукла оболонка
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    for i in range(len(hull_points)):
        start_point = hull_points[i]
        end_point = hull_points[(i + 1) % len(hull_points)]
        plt.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], 'g-', label='Convex Hull' if i == 0 else "")

    # Трикутник
    triangle = np.int0(triangle)
    for i in range(3):
        x = [triangle[i][0][0], triangle[(i+1) % 3][0][0]]
        y = [triangle[i][0][1], triangle[(i+1) % 3][0][1]]
        plt.plot(x, y, 'b-', label='Enclosing Triangle' if i == 0 else "")

    plt.legend()
    plt.axis('equal')
    plt.show()

    # Обчислення площі трикутника
    area = calculate_triangle_area(triangle)
    return triangle[:, 0, :], area

# Інтерактивний інтерфейс
def create_interface():
    output = widgets.Output()
    points = []

    def clear_all(_):
        with output:
            clear_output()
        points.clear()

    def on_manual_input(_):
        count = int(point_count_input.value)
        fields = []
        for i in range(count):
            fields.append(widgets.Text(description=f"Точка {i+1}:", layout=widgets.Layout(width='50%')))

        submit_button = widgets.Button(description="Побудувати", button_style='success')
        submit_button.on_click(lambda _: on_build([f.value for f in fields], 'manual'))
        display(*fields, submit_button)

    def on_random_input(_):
        count = int(point_count_input.value)
        points.clear()
        points.extend(np.random.randint(200, 600, size=(count, 2)))
        with output:
            clear_output()
            triangle, area = draw_enclosing_triangle_and_hull(np.array(points))
            if triangle is not None:
                print(f"Координати трикутника:\n{triangle}")
                print(f"Мінімальна площа трикутника: {area}")

    def on_build(data, mode):
        with output:
            clear_output()
            try:
                if mode == 'manual':
                    points.clear()
                    for d in data:
                        x, y = map(int, d.split())
                        points.append((x, y))
                triangle, area = draw_enclosing_triangle_and_hull(np.array(points))
                if triangle is not None:
                    print(f"Координати трикутника:\n{triangle}")
                    print(f"Мінімальна площа трикутника: {area}")
            except Exception as e:
                print(f"Помилка: {e}")

    manual_button = widgets.Button(description="Ввід вручну", button_style='info')
    manual_button.on_click(on_manual_input)

    random_button = widgets.Button(description="Рандомні точки", button_style='primary')
    random_button.on_click(on_random_input)

    clear_button = widgets.Button(description="Очистити", button_style='danger')
    clear_button.on_click(clear_all)

    point_count_input = widgets.IntText(description="Кількість точок:", value=10, layout=widgets.Layout(width='50%'))

    interface = widgets.VBox([
        widgets.HBox([manual_button, random_button, clear_button]),
        point_count_input,
        output
    ])

    display(interface)

# Запуск інтерфейсу
create_interface()